In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import pickle

C:\Users\aboub\PycharmProjects\search-engine\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4080'

### Import model from huggingFace

In [3]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
model.to("cuda")

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

### load data and remove duplicates

In [4]:
df = pd.read_csv('data/gutenberg_metadata.csv')

In [5]:
l_title = df.Title.tolist()
print(len(l_title))
l_title = list(set(l_title))
print(len(l_title))

15331
8766


## encode_data

In [6]:
title_embeddings = model.encode(l_title).astype('float32')
index = faiss.IndexFlatL2(title_embeddings.shape[1])
index.add(title_embeddings)

C:\Users\aboub\PycharmProjects\search-engine\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [7]:
index_to_title = {i: doc for i, doc in enumerate(l_title, 0)}

### Saving

In [8]:
faiss.write_index(index, "data/mock_data.index")

In [9]:
pickle.dump(index_to_title, open('data/mock_index_to_title.pkl', 'wb'))